## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-31-18-49-24 +0000,bbc,Head-on crash between police van and prison bu...,https://www.bbc.com/news/articles/c3v3g9z139wo...
1,2025-08-31-18-44-11 +0000,nypost,"AI isn’t coming for your job, says Trump labor...",https://nypost.com/2025/08/31/us-news/no-pink-...
2,2025-08-31-18-37-57 +0000,nyt,Hard-Nosed Sheriff Who Inspired ‘Walking Tall’...,https://www.nytimes.com/2025/08/31/us/buford-p...
3,2025-08-31-18-36-50 +0000,nyt,"Israel Says It Killed Abu Obeida, Spokesman fo...",https://www.nytimes.com/2025/08/31/world/middl...
4,2025-08-31-18-36-46 +0000,nypost,"US Open being impacted by NYC’s hot, sticky we...",https://nypost.com/2025/08/31/us-news/us-open-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,23
402,china,9
162,gaza,9
204,city,8
29,israel,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
11,2025-08-31-18-12-00 +0000,wsj,The Supreme Court has flirted with a flurry of...,https://www.wsj.com/politics/policy/trump-tari...,59
22,2025-08-31-17-11-30 +0000,nypost,"Trump says he, Putin and Zelensky will meet to...",https://nypost.com/2025/08/31/world-news/trump...,54
28,2025-08-31-16-15-56 +0000,nypost,Rep. Ilhan Omar slams Trump official for point...,https://nypost.com/2025/08/31/us-news/rep-ilha...,51
42,2025-08-31-14-53-00 +0000,wsj,Israel Says It Has Killed Hamas Spokesman in G...,https://www.wsj.com/world/middle-east/israel-s...,48
102,2025-08-31-07-40-59 +0000,nypost,Trump slams JB Pritzker on Chicago crime after...,https://nypost.com/2025/08/31/us-news/trump-sl...,48


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
11,59,2025-08-31-18-12-00 +0000,wsj,The Supreme Court has flirted with a flurry of...,https://www.wsj.com/politics/policy/trump-tari...
42,48,2025-08-31-14-53-00 +0000,wsj,Israel Says It Has Killed Hamas Spokesman in G...,https://www.wsj.com/world/middle-east/israel-s...
37,42,2025-08-31-15-15-17 +0000,nypost,Tennis star Karolína Muchová halts US Open mat...,https://nypost.com/2025/08/31/us-news/tennis-a...
96,35,2025-08-31-09-00-00 +0000,wsj,Democratic Party moderates see Virginia govern...,https://www.wsj.com/politics/elections/in-virg...
28,28,2025-08-31-16-15-56 +0000,nypost,Rep. Ilhan Omar slams Trump official for point...,https://nypost.com/2025/08/31/us-news/rep-ilha...
51,28,2025-08-31-13-28-31 +0000,nypost,Putin receives red carpet welcome in China for...,https://nypost.com/2025/08/31/world-news/putin...
67,25,2025-08-31-10-20-35 +0000,nypost,Montana bar shooting suspect Michael Paul Brow...,https://nypost.com/2025/08/31/us-news/montana-...
150,25,2025-08-30-19-00-34 +0000,nypost,Graphic video shows Israeli father throw himse...,https://nypost.com/2025/08/30/world-news/video...
113,24,2025-08-31-02-17-54 +0000,nypost,Lisa Cook standing up for the Federal Reserve ...,https://nypost.com/2025/08/30/business/lisa-co...
57,22,2025-08-31-11-48-40 +0000,nypost,Fecal contamination at US beaches hinders Labo...,https://nypost.com/2025/08/31/us-news/fecal-co...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
